# Atividade 1

## Word2Vec

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
documentos = [
    'o rei e um homem de bravura',
    'a rainha e uma mulher de sabedoria',
    'o menino e um homem jovem',
    'a menina e uma mulher jovem',
    'o principe e um jovem rei',
    'a princesa e uma jovem rainha',
    'principe e um menino que sera rei',
    'princesa e uma menina que sera rainha',
    'o homem e bruto',
    'a mulher e bonita'
]

In [ ]:
# remove stopwords

def remove_sw(documentos):
    stop_words = ['e', 'o', 'a', 'um', 'uma', 'sera', 'que', 'de']
    resultado = []
    for doc in documentos:
        tmp = doc.split(' ')
        for sw in stop_words:
            if sw in tmp:
                tmp.remove(sw)
        resultado.append(" ".join(tmp))
    
    return resultado

In [ ]:
# Filtra documentos
documentos = remove_sw(documentos)
documentos

In [ ]:
# Cria um conjunto de palavras (vocabulário)
vocabulario = []
for doc in documentos:
    for palavra in doc.split(' '):
        vocabulario.append(palavra)

vocabulario = set(vocabulario)
vocabulario

In [ ]:
# Crio um vetor representativo para cada palavra do vocabulário utilizando Skip-gram com 2 n-grams

label_palavra = {}
N = 2

for i, p in enumerate(vocabulario):
    label_palavra[p] = i

ss = []
for s in documentos:
    ss.append(s.split())

_dt = []
for s in ss:
    for index, p in enumerate(s):
        for vizinho in s[max(index - N, 0) : min(index + N, len(s)) + 1] : 
            if vizinho != p:
                _dt.append([p, vizinho])
                
_dt

In [ ]:
df = pd.DataFrame(_dt, columns = ['input', 'target'])
df

## Rede

In [ ]:
inputs = []
targets = []
OHD = len(vocabulario)

def one_hot_encoding(dp):
    ohe = np.zeros(OHD)
    ohe[dp] = 1
    return ohe

for i, j in zip(df['input'], df['target']):
    inputs.append(one_hot_encoding(label_palavra[i]))
    targets.append(one_hot_encoding(label_palavra[j]))

X_train = np.asarray(inputs)
Y_train = np.asarray(targets)


x = tf.placeholder(tf.float32, shape = (None, OHD))
y_label = tf.placeholder(tf.float32, shape = (None, OHD))

EMBEDDING = 2 

W_1 = tf.Variable(tf.random_normal([OHD, EMBEDDING]))
b_1 = tf.Variable(tf.random_normal([1]))
densa = tf.add(tf.matmul(x, W_1), b_1)

W_2 = tf.Variable(tf.random_normal([EMBEDDING, OHD]))
b_2 = tf.Variable(tf.random_normal([1]))
pred = tf.nn.softmax(tf.add( tf.matmul(densa, W_2), b_2))

loss = tf.reduce_mean(-tf.reduce_sum(y_label * tf.log(pred), axis = [1]))

train_op = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

In [ ]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init) 

iteracao = 25000
for i in range(iteracao):
    sess.run(train_op, feed_dict = {x: X_train, y_label: Y_train})
    if i % 5000 == 0:
        print('iteração {0} - loss {1}'.format(str(i), sess.run(loss, feed_dict={x: X_train, y_label: Y_train})))

In [ ]:
vetores = sess.run(W_1 + b_1)
vetores

In [ ]:
w2v_df = pd.DataFrame(vetores, columns = ['x', 'y'])
w2v_df['palavra'] = vocabulario
w2v_df = w2v_df[['palavra', 'x', 'y']]
w2v_df

In [ ]:
_, ax = plt.subplots(figsize = (15, 15))

for p, x, y in zip(w2v_df['palavra'], w2v_df['x'], w2v_df['y']):
    ax.annotate(p, (x, y))
    
_P = 1.0
ax_min = np.amin(vetores, axis = 0)[0] - _P
ax_max = np.amax(vetores, axis = 0)[0] + _P
ay_min = np.amin(vetores, axis = 0)[1] - _P
ay_max = np.amax(vetores, axis = 0)[1] + _P
 
plt.xlim(ax_min, ax_max)
plt.ylim(ay_min, ay_max)

## Desafio

Com a base de laudos fornecida, experimente utilizar a mesma lógica proposta acima para gerar uma rede através da biblioteca GenSim. 